In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").appName("Datasetexploration").getOrCreate()

23/04/07 20:19:49 WARN Utils: Your hostname, julian-Latitude-7280 resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface wlp1s0)
23/04/07 20:19:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/07 20:19:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/07 20:19:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

In [4]:
df = spark.read.option("header","true").csv("/home/julian/FORD_GO_BIKE_PROJECT/Dataset-2017-fordgobike-tripdata.csv")

In [5]:
df.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: string (nullable = true)
 |-- start_station_longitude: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: string (nullable = true)
 |-- end_station_longitude: string (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [6]:
df.count()

519700

In [7]:
df.select('member_gender').distinct().show()

+-------------+
|member_gender|
+-------------+
|         null|
|       Female|
|        Other|
|         Male|
+-------------+



In [8]:
df == df.dropna()

False

In [10]:
columns = df.columns

In [11]:
# finding the number of null values in specific columns

from pyspark.sql.functions import col, isnan, when, count
df.select([count(when((isnan(c)) | (col(c).isnull()), c)).alias(c) for c in columns]).show()

TypeError: 'Column' object is not callable

In [25]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+----------+--------+----------------+------------------+----------------------+-----------------------+--------------+----------------+--------------------+---------------------+-------+---------+-----------------+-------------+------+
|start_time|end_time|start_station_id|start_station_name|start_station_latitude|start_station_longitude|end_station_id|end_station_name|end_station_latitude|end_station_longitude|bike_id|user_type|member_birth_year|member_gender|pyment|
+----------+--------+----------------+------------------+----------------------+-----------------------+--------------+----------------+--------------------+---------------------+-------+---------+-----------------+-------------+------+
|         0|       0|               0|                 0|                     0|                      0|             0|               0|                   0|                    0|      0|        0|                0|            0|     0|
+----------+--------+----------------+--------------

In [9]:
df.columns

['start_time',
 'end_time',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'member_birth_year',
 'member_gender',
 'pyment']

In [13]:
df.filter(df['member_gender'].isna())

TypeError: 'Column' object is not callable

In [14]:
# import packages 
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types


def getNullNanBlankCount(df: DataFrame) -> DataFrame: 
  
  # get all the column names into a list
  allColsList = df.columns
  
  # create an empty list to temporarily store column names and count values
  listBuffer = []
  
  for field in allColsList:   
    # count nulls
    nullCondition = F.isnull(F.col(field))
    nullCount = df.select(F.col(field)).filter(nullCondition).count()  
    
    # count nans
    nanCondition = F.isnan(F.col(field))
    nanCount = df.select(F.col(field)).filter(nanCondition).count()  
    
    # count blank/empty values
    blankCount = df.select(F.col(field)).filter(F.col(field) == " ").count()  
     
    # append column names and count values to the list
    listBuffer.append((field, nullCount, nanCount, blankCount))

  # convert listBuffer into a new dataframe
  resDf = spark.createDataFrame(listBuffer, ["table_column_name", "null_count", "nan_count", "blank_count"])
 
  return resDf
  
  

In [ ]:
#sdf.transform(getNullNanBlankCount()).show(false)

In [15]:
df.transform(getNullNanBlankCount).show(truncate=False)

+-----------------------+----------+---------+-----------+
|table_column_name      |null_count|nan_count|blank_count|
+-----------------------+----------+---------+-----------+
|start_time             |0         |0        |0          |
|end_time               |0         |0        |0          |
|start_station_id       |0         |0        |0          |
|start_station_name     |0         |0        |0          |
|start_station_latitude |0         |0        |0          |
|start_station_longitude|0         |0        |0          |
|end_station_id         |0         |0        |0          |
|end_station_name       |0         |0        |0          |
|end_station_latitude   |0         |0        |0          |
|end_station_longitude  |0         |0        |0          |
|bike_id                |0         |0        |0          |
|user_type              |0         |0        |0          |
|member_birth_year      |66541     |0        |0          |
|member_gender          |66462     |0        |0         

In [18]:
df.select('pyment').distinct().show()

+-----------+
|     pyment|
+-----------+
| app wallet|
|credit card|
+-----------+



In [ ]:
df.select('pyment').distinct().show()

In [8]:
df.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: string (nullable = true)
 |-- start_station_longitude: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: string (nullable = true)
 |-- end_station_longitude: string (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



In [12]:
df.agg({'end_station_id':'mean'}).show()

+-------------------+
|avg(end_station_id)|
+-------------------+
|  92.18404079276506|
+-------------------+



In [8]:
df1 = df

In [7]:
df.select('start_time','end_time').count()

519700

In [18]:
df1.withColumn('distance_travelled', haversine((df['start_station_latitude'],df['start_station_longitude']),
                    (df['end_station_latitude'],df['end_station_longitude']),unit=Unit.NAUTICAL_MILES ))

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [16]:
pip install haversine

  Using cached haversine-2.8.0-py2.py3-none-any.whl (7.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
import haversine
from haversine import haversine, Unit

# New York City coordinates
nyc_coord = (40.7128, -74.0060)

# Los Angeles coordinates
la_coord = (34.0522, -118.2437)

# Calculate distance between NYC and LA
distance = haversine(nyc_coord, la_coord, unit=Unit.MILES)

print(f"Distance between NYC and LA: {distance:.1f} miles")


Distance between NYC and LA: 2445.6 miles


In [48]:
df2 = df.dropna()

In [49]:
df1 = df2

In [50]:
df1 = df1.withColumn('start_station_latitude',df['start_station_latitude'].cast("double")).\
         withColumn('start_station_longitude',df['start_station_longitude'].cast("double")).\
        withColumn('end_station_latitude',df['end_station_latitude'].cast("double")).\
        withColumn('end_station_longitude',df['end_station_longitude'].cast("double"))
        

In [73]:
from pyspark.sql.functions import udf
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType,DoubleType
# defining the function you want 
def distance(lat1,long1,lat2,long2):
    loc1 = (lat1,long1)
    loc2 = (lat2,long2)
    distance1 =haversine.haversine(loc1,loc2, Unit.KILOMETERS)
    return distance1

distanceUDF = udf(distance , DoubleType())



In [74]:
df1.withColumn("distance_travelled", distanceUDF(df['start_station_latitude'],df['start_station_longitude']\
                                                 ,df['end_station_latitude']\
                                                 ,df['end_station_longitude']))
                        

AnalysisException: Resolved attribute(s) start_station_latitude#20,start_station_longitude#21,end_station_latitude#24,end_station_longitude#25 missing from start_time#16,end_time#17,start_station_id#18,start_station_name#19,start_station_latitude#366,start_station_longitude#382,end_station_id#22,end_station_name#23,end_station_latitude#398,end_station_longitude#414,bike_id#26,user_type#27,member_birth_year#28,member_gender#29,pyment#30 in operator !Project [start_time#16, end_time#17, start_station_id#18, start_station_name#19, start_station_latitude#366, start_station_longitude#382, end_station_id#22, end_station_name#23, end_station_latitude#398, end_station_longitude#414, bike_id#26, user_type#27, member_birth_year#28, member_gender#29, pyment#30, distance(start_station_latitude#20, start_station_longitude#21, end_station_latitude#24, end_station_longitude#25) AS distance_travelled#536]. Attribute(s) with the same name appear in the operation: start_station_latitude,start_station_lo

In [53]:
df1.count()

453159

In [55]:
df1.select('start_station_latitude').show()

+----------------------+
|start_station_latitude|
+----------------------+
|           37.77643482|
|           37.78487208|
|            37.8688126|
|             37.866249|
|            37.7766392|
|            37.7766392|
|             37.795001|
|             37.795001|
|            37.7737172|
|           37.78588063|
|             37.333955|
|            37.7510171|
|           37.79801364|
|           37.79801364|
|           37.87055533|
|           37.80456235|
|            37.7662102|
|            37.3319323|
|            37.8090126|
|             37.765052|
+----------------------+
only showing top 20 rows



In [56]:
df1.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: double (nullable = true)
 |-- start_station_longitude: double (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: double (nullable = true)
 |-- end_station_longitude: double (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)

